<a href="https://colab.research.google.com/github/nouval0425/Learning-Python/blob/main/UTS_NOMOR_4_13182420050.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOMOR 4**

In [ ]:
import datetime

# --- 1. Class Entitas (Buku & Anggota) ---

class Buku:
    """Class untuk merepresentasikan satu unit buku."""
    def __init__(self, kode, judul, penulis):
        self.kode = kode
        self.judul = judul
        self.penulis = penulis
        self.status = "Tersedia" # Status bisa: Tersedia / Dipinjam

    def tampilkan_info(self):
        print(f"  [{self.kode}] {self.judul} oleh {self.penulis} (Status: {self.status})")

    def pinjam_buku(self):
        """Mengubah status buku menjadi 'Dipinjam'."""
        if self.status == "Tersedia":
            self.status = "Dipinjam"
            return True
        else:
            print(f"Error: Buku '{self.judul}' sedang dipinjam.")
            return False

    def kembalikan_buku(self):
        """Mengubah status buku menjadi 'Tersedia'."""
        self.status = "Tersedia"

class Anggota:
    """Class untuk merepresentasikan anggota perpustakaan."""
    def __init__(self, id_anggota, nama):
        self.id_anggota = id_anggota
        self.nama = nama
        self.buku_pinjaman = [] # List untuk mencatat buku yg sedang dipinjam

    def tampilkan_info(self):
        print(f"  [{self.id_anggota}] {self.nama} (Meminjam: {len(self.buku_pinjaman)} buku)")


# --- 2. Class Transaksi (Peminjaman & Denda) ---

class Peminjaman:

    def __init__(self, anggota, buku):
        self.anggota = anggota
        self.buku = buku
        self.tanggal_pinjam = datetime.date.today()
        self.denda_per_hari = 1000 # Denda standar

    def hitung_denda(self, hari_terlambat):
        """Menghitung denda standar."""
        if hari_terlambat <= 0:
            return 0
        denda = hari_terlambat * self.denda_per_hari
        print(f"Denda (Reguler): {hari_terlambat} hari x Rp {self.denda_per_hari:,} = Rp {denda:,}")
        return denda

    def tampilkan_info(self):
        print(f"  Peminjam: {self.anggota.nama} | Buku: {self.buku.judul}")


class PeminjamanKhusus(Peminjaman):
    """
    Class turunan untuk peminjaman khusus (misal: buku referensi).
    Meng-override method hitung_denda.
    """
    def __init__(self, anggota, buku, catatan):
        super().__init__(anggota, buku)
        self.catatan = catatan
        self.denda_per_hari = 5000 # Denda khusus lebih mahal

    # --- Method Overriding ---
    def hitung_denda(self, hari_terlambat):
        """
        Override method:
        Menghitung denda khusus yang tarifnya lebih tinggi.
        """
        if hari_terlambat <= 0:
            return 0
        denda = hari_terlambat * self.denda_per_hari
        print(f"Denda (Khusus): {hari_terlambat} hari x Rp {self.denda_per_hari:,} = Rp {denda:,}")
        return denda

    def tampilkan_info(self):
        super().tampilkan_info()
        print(f"    Catatan: {self.catatan}")


# --- 3. Class Manager (Perpustakaan) ---

class Perpustakaan:
    """
    Class utama yang mengelola 'list of objects'
    untuk buku, anggota, dan peminjaman.
    """
    def __init__(self):
        self.daftar_buku = []     # List of Buku objects
        self.daftar_anggota = []  # List of Anggota objects
        self.daftar_peminjaman = [] # List of Peminjaman objects

    def tambah_buku(self, kode, judul, penulis):
        buku_baru = Buku(kode, judul, penulis)
        self.daftar_buku.append(buku_baru)
        print(f"Buku '{judul}' ditambahkan ke koleksi.")

    def tambah_anggota(self, id_anggota, nama):
        anggota_baru = Anggota(id_anggota, nama)
        self.daftar_anggota.append(anggota_baru)
        print(f"Anggota '{nama}' terdaftar.")

    # --- Helper methods untuk mencari ---
    def cari_buku(self, kode):
        for buku in self.daftar_buku:
            if buku.kode == kode: return buku
        return None

    def cari_anggota(self, id_anggota):
        for anggota in self.daftar_anggota:
            if anggota.id_anggota == id_anggota: return anggota
        return None

    def buat_peminjaman(self, id_anggota, kode_buku, jenis_pinjaman="reguler"):
        anggota = self.cari_anggota(id_anggota)
        buku = self.cari_buku(kode_buku)

        if not anggota or not buku:
            print("Error: Anggota atau Buku tidak ditemukan.")
            return

        if buku.pinjam_buku(): # Jika buku berhasil dipinjam (status berubah)
            if jenis_pinjaman == "khusus":
                pinjaman = PeminjamanKhusus(anggota, buku, "Buku Referensi, denda tinggi")
            else:
                pinjaman = Peminjaman(anggota, buku)

            self.daftar_peminjaman.append(pinjaman)
            anggota.buku_pinjaman.append(buku)
            print(f"Transaksi berhasil: {anggota.nama} meminjam {buku.judul}.")

    def proses_pengembalian(self, kode_buku, hari_terlambat):
        buku = self.cari_buku(kode_buku)
        pinjaman_terpilih = None

        # Cari transaksi peminjaman berdasarkan kode buku
        for pinjaman in self.daftar_peminjaman:
            if pinjaman.buku.kode == kode_buku:
                pinjaman_terpilih = pinjaman
                break

        if not pinjaman_terpilih:
            print(f"Error: Tidak ada catatan peminjaman untuk buku (kode: {kode_buku}).")
            return

        # Hitung denda (Polimorfisme: method yg benar akan dipanggil)
        denda = pinjaman_terpilih.hitung_denda(hari_terlambat)

        # Selesaikan transaksi
        pinjaman_terpilih.buku.kembalikan_buku()
        pinjaman_terpilih.anggota.buku_pinjaman.remove(buku)
        self.daftar_peminjaman.remove(pinjaman_terpilih)

        print(f"Pengembalian '{buku.judul}' oleh {pinjaman_terpilih.anggota.nama} berhasil.")
        print(f"Total Denda: Rp {denda:,}")

    def tampilkan_status(self):
        print("\n--- Status Perpustakaan ---")
        print("Koleksi Buku:")
        for buku in self.daftar_buku: buku.tampilkan_info()
        print("\nDaftar Anggota:")
        for anggota in self.daftar_anggota: anggota.tampilkan_info()
        print("\nDaftar Peminjaman Aktif:")
        for pinjaman in self.daftar_peminjaman: pinjaman.tampilkan_info()


# --- 4. Contoh Penggunaan ---

if __name__ == "__main__":

    # 1. Setup perpustakaan
    perpus = Perpustakaan()
    perpus.tambah_buku("B001", "Dasar Pemrograman Python", "Guido van Rossum")
    perpus.tambah_buku("B002", "Struktur Data & Algoritma", "Thomas Cormen")
    perpus.tambah_buku("B003", "Kamus Besar Fisika", "Penerbit X") # Buku khusus

    perpus.tambah_anggota("A01", "Andi")
    perpus.tambah_anggota("A02", "Budi")

    perpus.tampilkan_status()
    print("\n" + "="*30 + "\n")

    # 2. Transaksi Peminjaman
    # Andi pinjam buku reguler
    perpus.buat_peminjaman("A01", "B001", "reguler")
    # Budi pinjam buku khusus
    perpus.buat_peminjaman("A02", "B003", "khusus")

    perpus.tampilkan_status()
    print("\n" + "="*30 + "\n")

    # 3. Transaksi Pengembalian (Demo Overriding Denda)

    # Andi mengembalikan (terlambat 2 hari)
    print("--- Andi Mengembalikan (Terlambat 2 Hari) ---")
    perpus.proses_pengembalian("B001", 2) # Denda standar

    print("\n" + "-"*30 + "\n")

    # Budi mengembalikan (terlambat 2 hari)
    print("--- Budi Mengembalikan (Terlambat 2 Hari) ---")
    perpus.proses_pengembalian("B003", 2) # Denda khusus (overridden)

    print("\n" + "="*30 + "\n")
    perpus.tampilkan_status()

Buku 'Dasar Pemrograman Python' ditambahkan ke koleksi.
Buku 'Struktur Data & Algoritma' ditambahkan ke koleksi.
Buku 'Kamus Besar Fisika' ditambahkan ke koleksi.
Anggota 'Andi' terdaftar.
Anggota 'Budi' terdaftar.

--- Status Perpustakaan ---
Koleksi Buku:
  [B001] Dasar Pemrograman Python oleh Guido van Rossum (Status: Tersedia)
  [B002] Struktur Data & Algoritma oleh Thomas Cormen (Status: Tersedia)
  [B003] Kamus Besar Fisika oleh Penerbit X (Status: Tersedia)

Daftar Anggota:
  [A01] Andi (Meminjam: 0 buku)
  [A02] Budi (Meminjam: 0 buku)

Daftar Peminjaman Aktif:


Transaksi berhasil: Andi meminjam Dasar Pemrograman Python.
Transaksi berhasil: Budi meminjam Kamus Besar Fisika.

--- Status Perpustakaan ---
Koleksi Buku:
  [B001] Dasar Pemrograman Python oleh Guido van Rossum (Status: Dipinjam)
  [B002] Struktur Data & Algoritma oleh Thomas Cormen (Status: Tersedia)
  [B003] Kamus Besar Fisika oleh Penerbit X (Status: Dipinjam)

Daftar Anggota:
  [A01] Andi (Meminjam: 1 buku)
  [A0